In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib as pyplot


In [9]:
column = ['userId','productId','Rating','timestamp']

In [10]:
df = pd.read_csv('ratings_Electronics.csv',names = column)

In [11]:
df.head()

,userId,productId,Rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [13]:
# Shape of the data
rows,columns = df.shape

In [22]:
print('Rows: {0}, Columns: {1} '.format(rows,columns))

Rows: 7824482, Columns: 4 


In [17]:
# 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824482 entries, 0 to 7824481
Data columns (total 4 columns):
userId       object
productId    object
Rating       float64
timestamp    int64
dtypes: float64(1), int64(1), object(2)
memory usage: 238.8+ MB


It looks like we have a dataset of 4 columns out of which 2 are object datatype. We may need to drill down and see what the data actually look like. 
Other two columns having one Float and one Int

In [20]:
# Check the Null values
df.isna().sum()

userId       0
productId    0
Rating       0
timestamp    0
dtype: int64

In [21]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Rating,7824482.0,4.012337e+00,1.380910e+00,1.0,3.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00
timestamp,7824482.0,1.338178e+09,6.900426e+07,912729600.0,1.315354e+09,1.361059e+09,1.386115e+09,1.406074e+09


Market_Basket_Optimisation.csv
MatrixFactorization.ipynb
PopularityBasedRS.ipynb
Project
Recommendation Systems_slides - Week2.pdf
Recommendation systems Problem Statement.pdf
SVD.ipynb
SVD.pdf
Untitled.code-workspace
apriori based recommendation.ipynb
content based recommendation_detailed.ipynb
cosine similarity example_demo
cosine similarity example_test_result
cosine similarity.ipynb
item-item collab filtering.ipynb
item-item cosine similarity.xlsx
links.csv
matrix factorization.xlsx
matrix factorization_detailed.ipynb
movies (2).csv
movies.csv
popularity based recommendations.ipynb
ratings (1).csv
ratings.csv
similarty measures.ipynb
tags.csv
user-user cosine similarity.xlsx
user-usercolabfilter.ipynb
